# Importing necassary libraries

In [1]:
import numpy as np
import scipy.io
from python_speech_features import mfcc
import matplotlib.pyplot as plt
import IPython.display as ipd
import os
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import soundfile as sf
import random
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
import joblib
import math
import statistics




# Defining the function that reads the audios from a given path using soundfile and returns 4 lists : audios, freqs, filepaths and problematic files for potential debugging

In [2]:

def read_audios(directory):
    audios = []
    freqs = []
    filepaths = []
    problematic_files = []
    
    # Walking through the directory that contains the dataset and reading each file that has the .wav extension
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.wav'):
                filepath = os.path.join(root, file)
                filepaths.append(filepath)
                try:
                    # Read the audio file using soundfile
                    data, freq = sf.read(filepath)
                    audios.append(data)
                    freqs.append(freq)
                except Exception as e:
                    print(f"Encountered an error with file: {filepath}. Error: {e}")
                    problematic_files.append(filepath)
    
    return audios, freqs, filepaths, problematic_files

# Example usage
audios, freqs, filepaths, problematic_files = read_audios(r'English_wav')


# Defining the function that extracts the mfcc features then removes the frames of silence finally it saves the mffc features into a .txt file according to gender 

In [3]:
def extractMfccs_RemoveSilence_saveMfccs(audios,freqs,filepaths, directory):
    mfccs = []
 
    for audio, freq, filepath in zip(audios, freqs,filepaths):
        # extract the MFCC features
        mfcc_features = mfcc(audio, freq, winlen=0.025, winstep=0.01, numcep=13, nfilt=26, nfft= 2048, lowfreq=0,
                         highfreq=None, preemph=0.97, ceplifter=22, appendEnergy=False)
        
        # calculate the energy
        energy = np.sum(mfcc_features**2, axis=1)
        # calculate the threshold for silence
        threshold = np.mean(energy) * 0.4
        #removing silence frames from mfccs
        voiced_indices = np.where(energy > threshold)[0]
        mfccs_voiced = mfcc_features[voiced_indices,:]
        mfccs.append(mfccs_voiced)
        
        # print the shape of the MFCCs before and after removing silence
        print(f"MFCCs before removing silence: {mfcc_features.shape}")
        print(f"MFCCs after removing silence: {mfccs_voiced.shape}")
        
       #saving mffcs 
       # extract the gender information from the file name
        gender = None
        if 'hommes' in filepath:
            gender = 'Hommes'
        elif 'femmes' in filepath:
            gender = 'Femmes'

        # save the MFCCs to the appropriate directory based on gender
        if gender is not None:
            gender_dir = os.path.join(directory, gender)
            if not os.path.exists(gender_dir):
                os.makedirs(gender_dir)
            mfcc_file = os.path.join(gender_dir,  os.path.splitext(os.path.basename(filepath))[0] + ".mfcc")
            np.savetxt(mfcc_file, mfccs_voiced, delimiter=',')
            
      
    
    return  mfccs

# Defining the function that splits the extraced mfccs into training and testing sets : 2/3 from male directory for training and 2/3 from female 

In [4]:
def train_test_split(mfcc_dir):
    # create separate lists for male and female file paths
    male_files = []
    female_files = []
    for root, dirs, files in os.walk(mfcc_dir):
        for file in files:
            if file.endswith('.mfcc'):
                if 'Hommes' in root:
                    male_files.append(os.path.join(root, file))
                elif 'Femmes' in root:
                    female_files.append(os.path.join(root, file))

    # shuffle the male and female lists independently
    random.shuffle(male_files)
    random.shuffle(female_files)

    # split the male and female lists into train and test based on the desired ratio
    male_train = male_files[:int(2/3*len(male_files))]
    male_test = male_files[int(2/3*len(male_files)):]

    female_train = female_files[:int(2/3*len(female_files))]
    female_test = female_files[int(2/3*len(female_files)):]

    
    
    # merge the train and test lists for both male and female
    train_files = male_train + female_train
    test_files = male_test + female_test

    # load the MFCC features from the saved files for the train and test sets
    train_mfccs = []
    test_mfccs = []

    for file in train_files:
        train_mfccs.append(np.loadtxt(file, delimiter=','))

    for file in test_files:
        test_mfccs.append(np.loadtxt(file, delimiter=','))

    # print the shapes of the train and test MFCC feature arrays
    print(f"Train male MFCCs shape: {np.array(male_train).shape}")
    print(f"Test male MFCCs shape: {np.array( male_test).shape}")
    print(f"Train female MFCCs shape: {np.array(female_train).shape}")
    print(f"Test female MFCCs shape: {np.array( female_test).shape}")
    print(f"Train MFCCs shape: {np.array(train_mfccs).shape}")
    print(f"Test MFCCs shape: {np.array(test_mfccs).shape}")
    
    return train_mfccs, test_mfccs


# Defining the functions that train the different GMM models and than save them as a pkl file

In [5]:
def gmm16(train_mfccs):
    # Initialize the GMM model with 16 classes
    gmm = GaussianMixture(n_components=16, covariance_type='diag', random_state=0)

    # Fit the GMM model to the training data
    gmm.fit(train_mfccs)
    
    # Save the trained GMM model to a file
    joblib.dump(gmm, r'gmm_model16_english.pkl')

    return gmm


In [6]:
def gmm32(train_mfccs):
    # Initialize the GMM model with 32 classes
    gmm = GaussianMixture(n_components=32, covariance_type='diag', random_state=0)

    # Fit the GMM model to the training data
    gmm.fit(train_mfccs)
    
    # Save the trained GMM model to a file
    joblib.dump(gmm, r'gmm_model32_english.pkl')

    return gmm

In [7]:
def gmm64(train_mfccs):
    # Initialize the GMM model with 64 classes
    gmm = GaussianMixture(n_components=64, covariance_type='diag', random_state=0)

    # Fit the GMM model to the training data
    gmm.fit(train_mfccs)
    
    # Save the trained GMM model to a file
    joblib.dump(gmm, r'gmm_model64_english.pkl')

    return gmm

In [8]:
def gmm128(train_mfccs):
    # Initialize the GMM model with 128 classes
    gmm = GaussianMixture(n_components=128, covariance_type='diag', random_state=0)

    # Fit the GMM model to the training data
    gmm.fit(train_mfccs)
    
    # Save the trained GMM model to a file
    joblib.dump(gmm, r'gmm_model128_english.pkl')

    return gmm

In [9]:
def gmm256(train_mfccs):
    # Initialize the GMM model with 254 classes
    gmm = GaussianMixture(n_components=256, covariance_type='diag', random_state=0)

    # Fit the GMM model to the training data
    gmm.fit(train_mfccs)
    
    # Save the trained GMM model to a file
    joblib.dump(gmm, r'gmm_model256_english.pkl')

    return gmm

# Getting the audios frequencies and filepaths from their directory using the function defined above

In [10]:
audios, freqs, filepaths, problematic_files = read_audios(r'English_wav')


# Extracting the mfcc features - Removing silence and saving the mfccs

In [11]:
mfccs = extractMfccs_RemoveSilence_saveMfccs(audios, freqs, filepaths,r'mfcc_english')

MFCCs before removing silence: (947, 13)
MFCCs after removing silence: (946, 13)
MFCCs before removing silence: (1467, 13)
MFCCs after removing silence: (1467, 13)
MFCCs before removing silence: (608, 13)
MFCCs after removing silence: (608, 13)
MFCCs before removing silence: (578, 13)
MFCCs after removing silence: (578, 13)
MFCCs before removing silence: (582, 13)
MFCCs after removing silence: (582, 13)
MFCCs before removing silence: (857, 13)
MFCCs after removing silence: (857, 13)
MFCCs before removing silence: (956, 13)
MFCCs after removing silence: (956, 13)
MFCCs before removing silence: (765, 13)
MFCCs after removing silence: (764, 13)
MFCCs before removing silence: (393, 13)
MFCCs after removing silence: (393, 13)
MFCCs before removing silence: (1643, 13)
MFCCs after removing silence: (1642, 13)
MFCCs before removing silence: (882, 13)
MFCCs after removing silence: (882, 13)
MFCCs before removing silence: (1188, 13)
MFCCs after removing silence: (1184, 13)
MFCCs before removing 

MFCCs before removing silence: (833, 13)
MFCCs after removing silence: (832, 13)
MFCCs before removing silence: (1048, 13)
MFCCs after removing silence: (1048, 13)
MFCCs before removing silence: (744, 13)
MFCCs after removing silence: (744, 13)
MFCCs before removing silence: (846, 13)
MFCCs after removing silence: (846, 13)
MFCCs before removing silence: (1697, 13)
MFCCs after removing silence: (1697, 13)
MFCCs before removing silence: (1074, 13)
MFCCs after removing silence: (1074, 13)
MFCCs before removing silence: (890, 13)
MFCCs after removing silence: (890, 13)
MFCCs before removing silence: (944, 13)
MFCCs after removing silence: (944, 13)
MFCCs before removing silence: (389, 13)
MFCCs after removing silence: (389, 13)
MFCCs before removing silence: (793, 13)
MFCCs after removing silence: (793, 13)
MFCCs before removing silence: (1461, 13)
MFCCs after removing silence: (1461, 13)
MFCCs before removing silence: (1210, 13)
MFCCs after removing silence: (1210, 13)
MFCCs before remov

MFCCs before removing silence: (472, 13)
MFCCs after removing silence: (472, 13)
MFCCs before removing silence: (389, 13)
MFCCs after removing silence: (389, 13)
MFCCs before removing silence: (266, 13)
MFCCs after removing silence: (266, 13)
MFCCs before removing silence: (442, 13)
MFCCs after removing silence: (442, 13)
MFCCs before removing silence: (201, 13)
MFCCs after removing silence: (201, 13)
MFCCs before removing silence: (591, 13)
MFCCs after removing silence: (591, 13)
MFCCs before removing silence: (273, 13)
MFCCs after removing silence: (273, 13)
MFCCs before removing silence: (527, 13)
MFCCs after removing silence: (527, 13)
MFCCs before removing silence: (521, 13)
MFCCs after removing silence: (521, 13)
MFCCs before removing silence: (806, 13)
MFCCs after removing silence: (806, 13)
MFCCs before removing silence: (173, 13)
MFCCs after removing silence: (173, 13)
MFCCs before removing silence: (392, 13)
MFCCs after removing silence: (392, 13)
MFCCs before removing silenc

MFCCs before removing silence: (517, 13)
MFCCs after removing silence: (517, 13)
MFCCs before removing silence: (850, 13)
MFCCs after removing silence: (850, 13)
MFCCs before removing silence: (2267, 13)
MFCCs after removing silence: (2267, 13)
MFCCs before removing silence: (313, 13)
MFCCs after removing silence: (313, 13)
MFCCs before removing silence: (1549, 13)
MFCCs after removing silence: (1548, 13)
MFCCs before removing silence: (713, 13)
MFCCs after removing silence: (713, 13)
MFCCs before removing silence: (482, 13)
MFCCs after removing silence: (482, 13)
MFCCs before removing silence: (334, 13)
MFCCs after removing silence: (334, 13)
MFCCs before removing silence: (283, 13)
MFCCs after removing silence: (283, 13)
MFCCs before removing silence: (194, 13)
MFCCs after removing silence: (194, 13)
MFCCs before removing silence: (559, 13)
MFCCs after removing silence: (559, 13)
MFCCs before removing silence: (508, 13)
MFCCs after removing silence: (508, 13)
MFCCs before removing si

MFCCs before removing silence: (349, 13)
MFCCs after removing silence: (349, 13)
MFCCs before removing silence: (424, 13)
MFCCs after removing silence: (424, 13)
MFCCs before removing silence: (286, 13)
MFCCs after removing silence: (286, 13)
MFCCs before removing silence: (290, 13)
MFCCs after removing silence: (290, 13)
MFCCs before removing silence: (883, 13)
MFCCs after removing silence: (883, 13)
MFCCs before removing silence: (571, 13)
MFCCs after removing silence: (571, 13)
MFCCs before removing silence: (401, 13)
MFCCs after removing silence: (401, 13)
MFCCs before removing silence: (497, 13)
MFCCs after removing silence: (497, 13)
MFCCs before removing silence: (391, 13)
MFCCs after removing silence: (391, 13)
MFCCs before removing silence: (277, 13)
MFCCs after removing silence: (276, 13)
MFCCs before removing silence: (325, 13)
MFCCs after removing silence: (325, 13)
MFCCs before removing silence: (957, 13)
MFCCs after removing silence: (957, 13)
MFCCs before removing silenc

...

MFCCs before removing silence: (985, 13)
MFCCs after removing silence: (985, 13)
MFCCs before removing silence: (670, 13)
MFCCs after removing silence: (664, 13)
MFCCs before removing silence: (593, 13)
MFCCs after removing silence: (593, 13)
MFCCs before removing silence: (816, 13)
MFCCs after removing silence: (815, 13)
MFCCs before removing silence: (1065, 13)
MFCCs after removing silence: (1061, 13)
MFCCs before removing silence: (743, 13)
MFCCs after removing silence: (743, 13)
MFCCs before removing silence: (2081, 13)
MFCCs after removing silence: (2075, 13)
MFCCs before removing silence: (645, 13)
MFCCs after removing silence: (643, 13)
MFCCs before removing silence: (569, 13)
MFCCs after removing silence: (567, 13)
MFCCs before removing silence: (1042, 13)
MFCCs after removing silence: (1041, 13)
MFCCs before removing silence: (283, 13)
MFCCs after removing silence: (282, 13)
MFCCs before removing silence: (639, 13)
MFCCs after removing silence: (638, 13)
MFCCs before removing 

MFCCs before removing silence: (397, 13)
MFCCs after removing silence: (393, 13)
MFCCs before removing silence: (1095, 13)
MFCCs after removing silence: (1093, 13)
MFCCs before removing silence: (334, 13)
MFCCs after removing silence: (334, 13)
MFCCs before removing silence: (455, 13)
MFCCs after removing silence: (454, 13)
MFCCs before removing silence: (726, 13)
MFCCs after removing silence: (722, 13)
MFCCs before removing silence: (300, 13)
MFCCs after removing silence: (299, 13)
MFCCs before removing silence: (657, 13)
MFCCs after removing silence: (652, 13)
MFCCs before removing silence: (301, 13)
MFCCs after removing silence: (300, 13)
MFCCs before removing silence: (315, 13)
MFCCs after removing silence: (315, 13)
MFCCs before removing silence: (2161, 13)
MFCCs after removing silence: (2159, 13)
MFCCs before removing silence: (384, 13)
MFCCs after removing silence: (382, 13)
MFCCs before removing silence: (739, 13)
MFCCs after removing silence: (737, 13)
MFCCs before removing si

MFCCs before removing silence: (701, 13)
MFCCs after removing silence: (700, 13)
MFCCs before removing silence: (758, 13)
MFCCs after removing silence: (758, 13)
MFCCs before removing silence: (1020, 13)
MFCCs after removing silence: (1020, 13)
MFCCs before removing silence: (936, 13)
MFCCs after removing silence: (936, 13)
MFCCs before removing silence: (1655, 13)
MFCCs after removing silence: (1655, 13)
MFCCs before removing silence: (1197, 13)
MFCCs after removing silence: (1197, 13)
MFCCs before removing silence: (989, 13)
MFCCs after removing silence: (989, 13)
MFCCs before removing silence: (986, 13)
MFCCs after removing silence: (986, 13)
MFCCs before removing silence: (581, 13)
MFCCs after removing silence: (581, 13)
MFCCs before removing silence: (1028, 13)
MFCCs after removing silence: (1028, 13)
MFCCs before removing silence: (2087, 13)
MFCCs after removing silence: (2087, 13)
MFCCs before removing silence: (1025, 13)
MFCCs after removing silence: (1025, 13)
MFCCs before rem

MFCCs before removing silence: (256, 13)
MFCCs after removing silence: (256, 13)
MFCCs before removing silence: (285, 13)
MFCCs after removing silence: (283, 13)
MFCCs before removing silence: (567, 13)
MFCCs after removing silence: (562, 13)
MFCCs before removing silence: (277, 13)
MFCCs after removing silence: (275, 13)
MFCCs before removing silence: (246, 13)
MFCCs after removing silence: (245, 13)
MFCCs before removing silence: (320, 13)
MFCCs after removing silence: (319, 13)
MFCCs before removing silence: (295, 13)
MFCCs after removing silence: (295, 13)
MFCCs before removing silence: (329, 13)
MFCCs after removing silence: (329, 13)
MFCCs before removing silence: (235, 13)
MFCCs after removing silence: (229, 13)
MFCCs before removing silence: (696, 13)
MFCCs after removing silence: (692, 13)
MFCCs before removing silence: (275, 13)
MFCCs after removing silence: (271, 13)
MFCCs before removing silence: (185, 13)
MFCCs after removing silence: (184, 13)
MFCCs before removing silenc

MFCCs before removing silence: (2856, 13)
MFCCs after removing silence: (2841, 13)
MFCCs before removing silence: (640, 13)
MFCCs after removing silence: (636, 13)
MFCCs before removing silence: (991, 13)
MFCCs after removing silence: (980, 13)
MFCCs before removing silence: (1072, 13)
MFCCs after removing silence: (1064, 13)
MFCCs before removing silence: (1392, 13)
MFCCs after removing silence: (1385, 13)
MFCCs before removing silence: (1283, 13)
MFCCs after removing silence: (1270, 13)
MFCCs before removing silence: (535, 13)
MFCCs after removing silence: (530, 13)
MFCCs before removing silence: (287, 13)
MFCCs after removing silence: (285, 13)
MFCCs before removing silence: (2085, 13)
MFCCs after removing silence: (2066, 13)
MFCCs before removing silence: (845, 13)
MFCCs after removing silence: (838, 13)
MFCCs before removing silence: (254, 13)
MFCCs after removing silence: (250, 13)
MFCCs before removing silence: (695, 13)
MFCCs after removing silence: (693, 13)
MFCCs before remov

## -----> We can see here that the size of the mfcc features has decreased after removing the frames ot silence

# Splitting into test and train sets according to gender

In [12]:
train_mfccs, test_mfccs = train_test_split(r'mfcc_english')

Train male MFCCs shape: (834,)
Test male MFCCs shape: (417,)
Train female MFCCs shape: (968,)
Test female MFCCs shape: (484,)
Train MFCCs shape: (1802,)
Test MFCCs shape: (901,)


C:\Users\HP\AppData\Local\Temp\ipykernel_2924\3397580047.py:45: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(f"Train MFCCs shape: {np.array(train_mfccs).shape}")
C:\Users\HP\AppData\Local\Temp\ipykernel_2924\3397580047.py:46: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(f"Test MFCCs shape: {np.array(test_mfccs).shape}")


# Stacking vertically the train and test MFCC features so that we can fit the gmm models

In [13]:
#stack vertically the train MFCC features 
mfcc_train = []
for train_mfcc in train_mfccs:
    mfcc_train.append(train_mfcc)
mfcc_train = np.concatenate(mfcc_train, axis=0)

#stack vertically the test MFCC features 
mfcc_test = []
for test_mfcc in test_mfccs:
    mfcc_test.append(test_mfcc)
mfcc_test = np.concatenate(mfcc_test, axis=0)


# Saving the test set into a txt file 

In [14]:
#Save the test mfccs in a file
test_mfccs = np.vstack(test_mfccs)
test_mfccs = np.array(test_mfccs, dtype=float)
np.savetxt(r'englishTest', test_mfccs, delimiter=',')



In [15]:
mfcc_train.shape

(1284399, 13)

# Training the different Gmm Models 

In [16]:
gmm16 = gmm16(mfcc_train)

In [17]:
gmm32 = gmm32(mfcc_train)

In [18]:
gmm64 = gmm64(mfcc_train)

In [19]:
gmm128= gmm128(mfcc_train)

In [20]:
gmm256= gmm256(mfcc_train)

# Evaluate the performance of each GMM model on the test set using the score_samples() function that returns an array containing the log-likelihood of each frame of the mfcc features

In [21]:
scores = []
for model in [gmm16, gmm32, gmm64, gmm128, gmm256]:
    score = model.score_samples(mfcc_test)
    scores.append(score)

# Print the scores
print('GMM16 score:', scores[0])
print('GMM32 score:', scores[1])
print('GMM64 score:', scores[2])
print('GMM128 score:', scores[3])
print('GMM256 score:', scores[4])



GMM16 score: [-61.41849654 -62.68619578 -62.29360203 ... -43.75833894 -44.66876686
 -44.86924497]
GMM32 score: [-60.1973371  -60.91531755 -60.02871252 ... -44.07424458 -44.83266365
 -45.22319779]
GMM64 score: [-59.84287657 -60.45427885 -60.14321655 ... -44.52200498 -45.16311388
 -45.42075026]
GMM128 score: [-60.36907749 -60.8655878  -60.40535513 ... -44.81467121 -45.41530834
 -45.28551998]
GMM256 score: [-60.28725831 -61.22874982 -61.37119225 ... -43.66774104 -45.44773416
 -45.43098419]


# Comparing the size of our mfcc_test set with the size of the scores array 

In [22]:
mfcc_test.shape

(648704, 13)

# Indeed the size of the scores array is the same 

In [23]:
 scores[0].shape

(648704,)

# In order to compare between the different GMM Models we need to calculate the score for the whole test set and we can do that by calculating the mean of the individual scores

In [24]:
#calculationg the score of the hole test set
print('GMM16 score:', scores[0].mean())
print('GMM32 score:', scores[1].mean())
print('GMM64 score:', scores[2].mean())
print('GMM128 score:', scores[3].mean())
print('GMM256 score:', scores[4].mean())


GMM16 score: -51.84626859685049
GMM32 score: -51.57585376265498
GMM64 score: -51.09029906523646
GMM128 score: -50.881848361960415
GMM256 score: -50.67077775228722


### From the results above we can see that the best score (the closest one to 0) is given by the model using 16 gaussians 